### Setup

In [ ]:
#download the image data
%cd /content/
!gdown https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
!unzip "nerf_data.zip"; rm "nerf_data.zip"
# !gdown https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
# !unzip davis.zip; rm davis.zip

In [ ]:
!git clone https://github.com/rohaldb/multi-mono-sf.git
%cd multi-mono-sf
!chmod -R +x ./
!pip install -r requirements.txt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Execution
Run the following and then look in `eval/kid_running`

In [ ]:
# Resize input images to be smaller resolution
%cd /content/multi-mono-sf/utils/
!python ./resize_images.py --data_path "/content/nerf_data/kid-running/dense/" --resize_height 288

In [ ]:
# Run mono-sf to obtain scene flow, induced optical flow and disparity estimates
%cd /content/multi-mono-sf/scripts
!./eval_custom.sh

In [10]:
!cp -r /content/multi-mono-sf/scripts/foo.zip /content/gdrive/MyDrive/Thesis/Data
!ls /content/gdrive/MyDrive/Thesis/Data